<a href="https://colab.research.google.com/github/mateusduarte-max/Projetos_Python/blob/main/02_Envio_parceiro_magalu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install workadays # pacote para calculo data prometida e data limite postagem

import pandas as pd
import re
import datetime
from workadays import workdays as wd


# Bibliotecas Google Golab
from google.colab import auth
auth.authenticate_user()

# Importar biblioteca biqquery
from google.cloud import bigquery

# Bibliotecas Google Sheet
import gspread
from oauth2client.client import GoogleCredentials
from gspread_dataframe import set_with_dataframe
gc = gspread.authorize(GoogleCredentials.get_application_default())

# Montar Drive
from google.colab import drive
drive.mount('/content/drive')

# Modulo colab
!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/modulos_colab.py /content


# Importar biblioteca biqquery
from modulos_colab import consulta_bigquery, grava_tabela_drive, busca_tabela_drive
#'maga-bigdata'
#'marketplace-analytics-333712'

Mounted at /content/drive


In [ ]:
df = consulta_bigquery('maga-bigdata', """SELECT DISTINCT

      DATETIME_ADD(CAST(o.created_at AS DATETIME), INTERVAL -3 HOUR) as data_captado,
      concat("LU-",o.order_id) as order_id,
      so.sub_order_id , 
      so.seller_id , 
      s.description as ultimo_status,
      date(f.deadline_date) as data_prometida_entrega,
      me.delivery_option_id as opcao_entrega,
      case when entrega.data_entrega is null then 'N' else 'S' end as entrega_realizada,
      entrega.data_entrega,
      IF (((date(f.deadline_date) >= date(entrega.data_entrega))),'S','N') as entrega_no_prazo,
      ss.Assistido,
      ss.Mundo,
      c.status as status_integra,
      date(c.shipped_carrier_date) as data_expedicao,
      f.deadline_value as dias_prazo,
      aprovacao.data_aprovacao

    
      
            FROM maga-bigdata.maestro.order as o
            left join maga-bigdata.maestro.sub_order as so on so.order_uuid = o.uuid
            left join maga-bigdata.maestro.sales_channel as sc on o.sales_channel_id = sc.id
            left join maga-bigdata.maestro.customer as cus on o.customer_uuid = cus.uuid
            left join maga-bigdata.maestro.status as s on so.current_status_id = s.id
            left join maga-bigdata.maestro.shipping f ON so.uuid = f.sub_order_uuid
            left join maga-bigdata.magaluentregas.shipping as me on me.order_id = o.order_id and so.seller_id = me.seller
            left join maga-bigdata.bi_adm.tb_safra_seller as ss on ss.Seller_MarketPlace_ID = so.seller_id
            left join maga-bigdata.integra.order as a on a.idsite = o.basket_id
            left join maga-bigdata.integra.business b on a.business_id = b.id and so.seller_id = b.SellerIdOpenApi
            left join maga-bigdata.integra.order_status c ON a.id = c.order_id_fk and c.Business_Id = b.id
            left join (SELECT cast(datetime_add(cast(min(date) AS datetime), interval -3 hour)as date) as data_entrega, sub_order_uuid
                FROM `maga-bigdata.maestro.event`
                where status_id  in (10,21)
                group by sub_order_uuid) as entrega on so.uuid = entrega.sub_order_uuid
            left join (SELECT cast(datetime_add(cast(min(date) AS datetime), interval -3 hour)as date) as data_aprovacao, sub_order_uuid
                FROM `maga-bigdata.maestro.event`
                where status_id  IN (2, 143, 142, 144, 148)
                group by sub_order_uuid) as aprovacao on so.uuid = aprovacao.sub_order_uuid


            where sc.organization_uuid = '4da25f48-4193-45de-b4ed-9b8c93b7f987'
                  and if(lower(cus.email) like '%@c2f.com%',0,1) = 1
                  and if(cus.email = 'treinamento@luizalabs.com',0,1) = 1
                  and cast(DATETIME_ADD(CAST(o.created_at AS DATETIME), INTERVAL -3 HOUR) as date) between '2021-12-01' and current_date()
                  --and EXTRACT(YEAR FROM date(o.created_at)) = 2022
                  --and EXTRACT(MONTH FROM date(o.created_at)) = 01
                  --and date(f.deadline_date) is null
                  and if(o.order_id = me.order_id,0,1) = 1
                  and so.seller_id <> 'magazineluiza'
                  --and so.seller_id = 'sostudo' 
                  --and s.description = 'Entregue'
                  --and o.order_id = '1112370509565158'
                  and s.description not in ('Cancelado', 'Cancelado por dados não confirmados', 'Cancelado Confirmada')
                  --and if(s.description = 'Cancelado',0,1) = 1
                  and c.status not in ('CANCELED')""")

In [ ]:
# Pedidos sem data de aprovação
pedidos_dataAprovacao_embranco = df.query('data_aprovacao=="" & data_captado>="2022-01-01"')
#pedidos_dataAprovacao_embranco['order_id'].count()
# Gravar pedidos na tabela (Drive)
grava_tabela_drive('Pedidos_análise', pedidos_dataAprovacao_embranco)

In [ ]:
# Ajuste na base
df['dias_prazo'] = df['dias_prazo'].replace('',0)
df['dias_prazo'] = df['dias_prazo'].astype(int)
df['data_prometida_entrega'] = df['data_prometida_entrega'].astype(object)
df['data_prometida_entrega'] = pd.to_datetime(df['data_prometida_entrega'])
df['mes_data_prometida_entrega'] = df['data_prometida_entrega'].dt.month
df['ano_data_prometida_entrega'] = df['data_prometida_entrega'].dt.year

df['data_aprovacao'] = df['data_aprovacao'].astype(object)
df['data_aprovacao'] = pd.to_datetime(df['data_aprovacao'])

base_2 = df.query("data_aprovacao!=''")

# Data Prometida v2
def data_prometida(base_2):
  if base_2['data_prometida_entrega'] =="":
    return wd.workdays(base_2['data_aprovacao'], base_2['dias_prazo'], country=None)
  else:
    return base_2['data_prometida_entrega']
  
# Criar coluna data prometida v2 , mês e ano
base_2['DATA_PROMETIDA_v2'] = base_2.apply(data_prometida, axis=1)
base_2['mes_data_prometida_v2'] = base_2['DATA_PROMETIDA_v2'].dt.month
base_2['ano_data_prometida_v2'] = base_2['DATA_PROMETIDA_v2'].dt.year

# Filtro meses
janeiro_2022 = base_2.query('ano_data_prometida_v2==2022 & mes_data_prometida_v2==1')
fevereiro_2022 = base_2.query('ano_data_prometida_v2==2022 & mes_data_prometida_v2==2')
# Contar pedidos fevereiro
fevereiro_2022['order_id'].count()
# Exportar para tabela BigQuery
fevereiro_2022.to_gbq(destination_table='marketplace_analytics.envio_parceiro_magalu_fevereiro2022_novo',project_id='maga-bigdata', if_exists='replace')
print("Tabela atualizada!")

In [ ]:
fevereiro_2022['order_id'].count()

In [ ]:
janeiro_2022['order_id'].count()